In [1]:
!pip install codecarbon

In [2]:
!pip install thop

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
import time
import os
import subprocess
import numpy as np
from tqdm import tqdm
import codecarbon
from thop import profile

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the transforms for the dataset with additional augmentations
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load the CIFAR-10 dataset
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split the full training dataset into training and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

# Create DataLoaders for the train, validation, and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Print dataset sizes for debugging
print(f"Training set size: {len(train_dataset)}")
print(f"Validation set size: {len(val_dataset)}")
print(f"Test set size: {len(test_dataset)}")

# Load the pretrained VGG16 model and modify the classifier
model = models.vgg16(pretrained=True)
input_lastLayer = model.classifier[6].in_features
model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),  # Increased dropout for regularization
    nn.Linear(input_lastLayer, 10)
)
model = model.to(device)

# Define loss function and optimizer with learning rate scheduler
learning_rate = 0.01
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9, weight_decay=5e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=3, verbose=True)

# Tracking metrics for analysis
train_losses, val_losses = [], []
train_accuracies, val_accuracies = [], []

# Use CodeCarbon for system-wide energy consumption tracking
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Function to get GPU power consumption using nvidia-smi
def get_gpu_power():
    try:
        # Query power.draw from nvidia-smi
        result = subprocess.run(
            ['nvidia-smi', '--query-gpu=power.draw', '--format=csv,nounits,noheader'],
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        power_draws = result.stdout.strip().split('\n')
        power_draws = [float(p) for p in power_draws]
        avg_power_draw = sum(power_draws) / len(power_draws)  # Average power across GPUs
        return avg_power_draw  # in watts
    except Exception as e:
        print(f"Error getting GPU power: {e}")
        return 0.0

# Tracking GPU-specific energy consumption
gpu_power_readings = []

# Training loop with GPU power monitoring
epochs = 30
start_training_time = time.time()

for epoch in range(epochs):
    # Training phase
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        images, labels = images.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Calculate training accuracy
        _, predicted = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

        # Record GPU power usage
        gpu_power_readings.append(get_gpu_power())

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct_train / total_train

    # Append metrics for plotting
    train_losses.append(train_loss)
    train_accuracies.append(train_accuracy)

    # Validation phase
    model.eval()
    running_val_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            running_val_loss += loss.item()

            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_loss = running_val_loss / len(val_loader)
    val_accuracy = 100 * correct_val / total_val

    # Append metrics for plotting
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%")
    print(f"Epoch [{epoch+1}/{epochs}], Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

    # Adjust learning rate based on validation loss
    scheduler.step(val_loss)

training_time = time.time() - start_training_time

# Stop the CodeCarbon tracker after training
emissions = tracker.stop()

# Calculate average GPU power and total GPU energy consumption (in kWh)
avg_gpu_power = sum(gpu_power_readings) / len(gpu_power_readings) if gpu_power_readings else 0.0
energy_consumption_gpu = (avg_gpu_power * training_time) / 3600  # Convert to kWh

# Save the model after training
model_path = "trained_vgg16_model.pth"
torch.save(model.state_dict(), model_path)

# Model analysis metrics
param_count = sum(p.numel() for p in model.parameters() if p.requires_grad)
torch.save(model.state_dict(), "temp_model.pth")
model_size = os.path.getsize("temp_model.pth") / (1024 * 1024)  # Convert bytes to MB
os.remove("temp_model.pth")

# FLOPs calculation
dummy_input = torch.randn(1, 3, 32, 32).to(device)
flops, _ = profile(model, inputs=(dummy_input,), verbose=False)

# Inference time calculation
model.eval()
inference_start = time.time()
with torch.no_grad():
    for _ in range(100):
        _ = model(dummy_input)
inference_time = (time.time() - inference_start) / 100

# Final energy and emissions report
print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Inference Time: {inference_time:.6f} seconds")

print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions (CodeCarbon): {emissions:.6f} kg")
print(f"Average GPU Power Consumption: {avg_gpu_power:.2f} W")
print(f"Total GPU Energy Consumption: {energy_consumption_gpu:.6f} kWh")


Using device: cuda


100%|██████████| 170M/170M [00:05<00:00, 29.8MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_We

Training set size: 40000
Validation set size: 10000
Test set size: 10000


...

Epoch [28/30], Train Loss: 0.1335, Train Accuracy: 95.53%
Epoch [28/30], Val Loss: 0.3214, Val Accuracy: 90.35%


Epoch 29/30 - Training:  13%|█▎        | 80/625 [00:09<00:49, 11.00it/s][codecarbon INFO @ 23:30:03] Energy consumed for RAM : 0.002770 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:30:03] Energy consumed for all GPUs : 0.035437 kWh. Total GPU Power : 52.32882678942569 W
[codecarbon INFO @ 23:30:03] Energy consumed for all CPUs : 0.024797 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:30:03] 0.063005 kWh of electricity used since the beginning.
Epoch 29/30 - Training:  36%|███▌      | 224/625 [00:24<00:37, 10.72it/s][codecarbon INFO @ 23:30:18] Energy consumed for RAM : 0.002790 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:30:18] Energy consumed for all GPUs : 0.035704 kWh. Total GPU Power : 64.04243157473682 W
[codecarbon INFO @ 23:30:18] Energy consumed for all CPUs : 0.024975 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:30:18] 0.063469 kWh of electricity used since the beginning.
Epoch 29/30 - Training:  59%|█████▊    | 366/625 [00:39<00:24, 

Epoch [29/30], Train Loss: 0.1223, Train Accuracy: 95.89%
Epoch [29/30], Val Loss: 0.3175, Val Accuracy: 90.03%


Epoch 30/30 - Training:  10%|█         | 65/625 [00:06<00:52, 10.73it/s][codecarbon INFO @ 23:31:18] Energy consumed for RAM : 0.002869 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:31:18] Energy consumed for all GPUs : 0.036700 kWh. Total GPU Power : 49.75979124072567 W
[codecarbon INFO @ 23:31:18] Energy consumed for all CPUs : 0.025683 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:31:18] 0.065252 kWh of electricity used since the beginning.
Epoch 30/30 - Training:  34%|███▍      | 215/625 [00:21<00:37, 11.07it/s][codecarbon INFO @ 23:31:33] Energy consumed for RAM : 0.002889 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:31:33] Energy consumed for all GPUs : 0.036967 kWh. Total GPU Power : 64.11063216360256 W
[codecarbon INFO @ 23:31:33] Energy consumed for all CPUs : 0.025860 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:31:33] 0.065716 kWh of electricity used since the beginning.
Epoch 30/30 - Training:  58%|█████▊    | 361/625 [00:36<00:25, 

Epoch [30/30], Train Loss: 0.1172, Train Accuracy: 96.07%
Epoch [30/30], Val Loss: 0.3108, Val Accuracy: 90.54%

--- Model Analysis ---
Parameter Count: 134301514
Model Size: 512.33 MB
FLOPs: 0.43 GFLOPs
Training Time: 2244.93 seconds
Average Inference Time: 0.003616 seconds

--- Energy and Emissions Report ---
CO2 Emissions (CodeCarbon): 0.018010 kg
Average GPU Power Consumption: 63.72 W
Total GPU Energy Consumption: 39.733080 kWh


In [4]:
import time
import os
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision import models, datasets, transforms
from tqdm import tqdm
import codecarbon
from torch.profiler import profile, ProfilerActivity

# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define dataset transforms
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomCrop(32, padding=4),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load datasets
full_train_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

# Split dataset
train_size = int(0.8 * len(full_train_dataset))
val_size = len(full_train_dataset) - train_size
train_dataset, val_dataset = random_split(full_train_dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4, pin_memory=True)

# Load teacher model
teacher_model = models.vgg16(pretrained=False)
teacher_model.classifier[6] = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(4096, 10)
)
teacher_model.load_state_dict(torch.load("trained_vgg16_model.pth"))
teacher_model = teacher_model.to(device)
teacher_model.eval()

# Define student model
student_model = models.mobilenet_v2(pretrained=True)
student_model.classifier[1] = nn.Sequential(
    nn.Linear(student_model.last_channel, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 10)
)
student_model = student_model.to(device)

# Define distillation loss
class DistillationLoss(nn.Module):
    def __init__(self, temperature=3.0, alpha=0.5):
        super(DistillationLoss, self).__init__()
        self.temperature = temperature
        self.alpha = alpha
        self.criterion_ce = nn.CrossEntropyLoss()

    def forward(self, student_logits, teacher_logits, labels):
        distillation_loss = nn.KLDivLoss(reduction='batchmean')(
            torch.log_softmax(student_logits / self.temperature, dim=1),
            torch.softmax(teacher_logits / self.temperature, dim=1)
        ) * (self.temperature ** 2)
        student_loss = self.criterion_ce(student_logits, labels)
        return self.alpha * distillation_loss + (1 - self.alpha) * student_loss

criterion = DistillationLoss(temperature=3.0, alpha=0.5)

# Define optimizer and scheduler
learning_rate = 0.001
optimizer = optim.AdamW(student_model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

# Track energy consumption
tracker = codecarbon.EmissionsTracker()
tracker.start()

# Training loop
epochs = 20
start_time = time.time()
gpu_power_readings = []

for epoch in range(epochs):
    student_model.train()
    running_loss = 0.0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
        student_outputs = student_model(inputs)
        loss = criterion(student_outputs, teacher_outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    scheduler.step()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss / len(train_loader):.4f}")

end_time = time.time()
training_time = end_time - start_time

# Stop CodeCarbon tracker
emissions = tracker.stop()

# Save model
torch.save(student_model.state_dict(), "student_model_mobilenetv2.pth")

# Model analysis
def get_model_size(model):
    torch.save(model.state_dict(), "temp_model.pth")
    size_mb = os.path.getsize("temp_model.pth") / (1024 * 1024)
    os.remove("temp_model.pth")
    return size_mb

def measure_inference_time(model, dataloader):
    model.eval()
    with torch.no_grad():
        start_time = time.time()
        for inputs, _ in dataloader:
            inputs = inputs.to(device)
            _ = model(inputs)
        end_time = time.time()
    return (end_time - start_time) / len(dataloader)

# Calculate FLOPs using torch.profiler
with profile(activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA]) as prof:
    dummy_input = torch.randn(1, 3, 32, 32).to(device)
    student_model(dummy_input)

flops = sum(event.cpu_time_total for event in prof.key_averages())

# Average GPU power consumption
avg_gpu_power = sum(gpu_power_readings) / len(gpu_power_readings) if gpu_power_readings else 0.0
total_energy_consumption = (avg_gpu_power * training_time) / 3600  # kWh

# Calculate and display final metrics
param_count = sum(p.numel() for p in student_model.parameters() if p.requires_grad)
model_size_mb = get_model_size(student_model)
inference_time_ms = measure_inference_time(student_model, test_loader) * 1000  # ms

print("\n--- Model Analysis ---")
print(f"Parameter Count: {param_count}")
print(f"Model Size: {model_size_mb:.2f} MB")
print(f"FLOPs: {flops / 1e9:.2f} GFLOPs")
print(f"Training Time: {training_time:.2f} seconds")
print(f"Average Inference Time: {inference_time_ms:.6f} ms")

print("\n--- Energy and Emissions Report ---")
print(f"CO2 Emissions (CodeCarbon): {emissions:.6f} kg")
print(f"Average GPU Power Consumption: {avg_gpu_power:.2f} W")
print(f"Total GPU Energy Consumption: {total_energy_consumption:.6f} kWh")

Using device: cuda
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
<ipython-input-4-a051c6344052>:50: F

Epoch [1/20], Loss: 3.6655


Epoch 2/20 - Training:   1%|          | 7/625 [00:01<01:10,  8.72it/s][codecarbon INFO @ 23:33:44] Energy consumed for RAM : 0.000079 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:33:44] Energy consumed for all GPUs : 0.000735 kWh. Total GPU Power : 45.79081895677121 W
[codecarbon INFO @ 23:33:44] Energy consumed for all CPUs : 0.000709 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:33:44] 0.001523 kWh of electricity used since the beginning.
Epoch 2/20 - Training:  28%|██▊       | 178/625 [00:16<00:31, 14.20it/s][codecarbon INFO @ 23:33:59] Energy consumed for RAM : 0.000099 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:33:59] Energy consumed for all GPUs : 0.000929 kWh. Total GPU Power : 46.781314114762765 W
[codecarbon INFO @ 23:33:59] Energy consumed for all CPUs : 0.000886 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:33:59] 0.001915 kWh of electricity used since the beginning.
Epoch 2/20 - Training:  56%|█████▌    | 348/625 [00:31<00:21, 12.

Epoch [2/20], Loss: 2.7542


...

Epoch 19/20 - Training:  23%|██▎       | 142/625 [00:13<00:35, 13.72it/s][codecarbon INFO @ 23:49:45] Energy consumed for RAM : 0.001346 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:49:45] Energy consumed for all GPUs : 0.013013 kWh. Total GPU Power : 47.22266973078153 W
[codecarbon INFO @ 23:49:45] Energy consumed for all CPUs : 0.012047 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:49:45] 0.026406 kWh of electricity used since the beginning.
Epoch 19/20 - Training:  50%|█████     | 314/625 [00:28<00:24, 12.50it/s][codecarbon INFO @ 23:50:00] Energy consumed for RAM : 0.001366 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:50:00] Energy consumed for all GPUs : 0.013212 kWh. Total GPU Power : 47.735671088930154 W
[codecarbon INFO @ 23:50:00] Energy consumed for all CPUs : 0.012224 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:50:00] 0.026802 kWh of electricity used since the beginning.
Epoch 19/20 - Training:  77%|███████▋  | 483/625 [00:43<00:11

Epoch [19/20], Loss: 1.7053


Epoch 20/20 - Training:   4%|▍         | 27/625 [00:02<00:48, 12.40it/s][codecarbon INFO @ 23:50:30] Energy consumed for RAM : 0.001405 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:50:30] Energy consumed for all GPUs : 0.013594 kWh. Total GPU Power : 46.24896001087872 W
[codecarbon INFO @ 23:50:30] Energy consumed for all CPUs : 0.012579 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:50:30] 0.027578 kWh of electricity used since the beginning.
Epoch 20/20 - Training:  31%|███       | 195/625 [00:17<00:32, 13.11it/s][codecarbon INFO @ 23:50:45] Energy consumed for RAM : 0.001425 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 23:50:45] Energy consumed for all GPUs : 0.013785 kWh. Total GPU Power : 45.99967895186482 W
[codecarbon INFO @ 23:50:45] Energy consumed for all CPUs : 0.012756 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 23:50:45] 0.027966 kWh of electricity used since the beginning.
[codecarbon INFO @ 23:50:45] 0.006937 g.CO2eq/s mean an estimat

Epoch [20/20], Loss: 1.7211


ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 576, 1, 1])

In [5]:
student_model.eval()
correct_test = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = student_model(images)
        _, predicted = torch.max(outputs, 1)
        total_test += labels.size(0)
        correct_test += (predicted == labels).sum().item()

test_accuracy = 100 * correct_test / total_test
print(f"Student Model Test Accuracy: {test_accuracy:.2f}%")

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


Student Model Test Accuracy: 78.43%
